Play with accessing DuckDB and Azure Storage Blobs using JupySQL Jupyter magics

In [ ]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = 0
%config SqlMagic.displaycon = False

In [9]:
import utils
from azure.identity import ChainedTokenCredential, ManagedIdentityCredential, InteractiveBrowserCredential

In [10]:
tenant_id='3a2d9a5b-25c0-47da-9dff-842d10755773'
account_name = "azduckdb"
container_name = "data"
blob_path = "HuggingFaceFW/fineweb/CC-MAIN-2013-20/*.parquet"

In [11]:
credential = ChainedTokenCredential(ManagedIdentityCredential(), InteractiveBrowserCredential(tenant_id=tenant_id))
connection_string = utils.get_connection_string(credential, account_name, container_name)

In [3]:
%sql duckdb:///:memory:

In [14]:
%%sql CREATE OR REPLACE SECRET {{account_name}} (
    TYPE AZURE,
    CONNECTION_STRING '{{connection_string}}',
    SCOPE 'az://{{account_name}}.blob.core.windows.net/{{container_name}}/'
)

,Success


In [15]:
%sql FROM duckdb_secrets()

,name,type,provider,persistent,storage,scope,secret_string
0,azduckdb,azure,config,False,memory,[az://azduckdb.blob.core.windows.net/data/],name=azduckdb;type=azure;provider=config;seria...


In [18]:
%sql SELECT COUNT(*) FROM 'az://{{account_name}}.blob.core.windows.net/{{container_name}}/{{blob_path}}'

,count_star()
0,5433000
